In [1]:
import time
import sys
import os
from bs4 import BeautifulSoup
import requests
import torch
from torch import nn, optim,utils
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as TF
from torch.utils.data import Subset
from collections import OrderedDict
from PIL import Image
import numpy as np

payload={'from':'/bbs/Beauty/index.html','yes':'yes'} 
rs=requests.session()
time.sleep(3)
res=rs.post('https://www.ptt.cc/ask/over18',data=payload) #把payload塞到cookie裏
url = "https://www.ptt.cc/bbs/Beauty/index2745.html" #大概前幾頁的index
r = rs.get(url)
content = r.text #content 是網頁原始碼
a=1
objSoup=BeautifulSoup(content, 'lxml')
#抓到2019的第一篇
while a==1:
    div_page=objSoup.find('div',"btn-group btn-group-paging")
    next_page=div_page.find_all('a')[2]['href']
    ppturl="https://www.ptt.cc"
    print(next_page)
    r = rs.get(ppturl+next_page)
    objSoup=BeautifulSoup(r.text, 'lxml')
    pttdivs=objSoup.find_all('div','r-ent')
    for p in pttdivs:
        if p.find('a') and p.find('a')['href']== '/bbs/Beauty/M.1546272669.A.765.html':
            a=0
index_page=ppturl+next_page

/bbs/Beauty/index2746.html
/bbs/Beauty/index2747.html
/bbs/Beauty/index2748.html


In [ ]:
#把題目資訊擷取下來
def catch_info(content):
    articles_list=[]
    articles_popular_list=[]
    objSoup=BeautifulSoup(content, 'lxml')
    pttdivs=objSoup.find_all('div','r-ent')
    for p in pttdivs:
            if p.find('a') and '公告' not in p.find('a').text:
                date=p.find('div','date').text
                title=p.find('a').text
                href=p.find('a')['href']
                articles_list.append({'date':''.join(date.split('/')),'title':title,'href':href})
                if p.find('div','nrec').text=='爆':
                    date=p.find('div','date').text
                    title=p.find('a').text
                    href=p.find('a')['href']
                    articles_popular_list.append({'date':''.join(date.split('/')),'title':title,'href':href})
    return articles_list,articles_popular_list,objSoup


In [ ]:
#抓2019左右的文章
def crawl():
    b=1
    articles=[]
    articles_popular=[]
    global index_page
    while b==1:
        r=rs.get(index_page)
        articles_list,articles_popular_list,objSoup=catch_info(r.content)
        articles.extend(articles_list) 
        articles_popular.extend(articles_popular_list)
        div_page=objSoup.find('div',"btn-group btn-group-paging")
        next_page=div_page.find_all('a')[2]['href']
        index_page=ppturl+next_page
        if len(articles)>1500 and int(articles[-1]['date'])<117:
            b=0
    #切下2019的範圍
    articles_final=articles[articles.index({'date': ' 101', 'title': '[神人] 雲林斗六跨年晚會', 'href': '/bbs/Beauty/M.1546272669.A.765.html'}):articles.index({'date': '1231',
  'title': '[正妹] 2019最後奶特',
  'href': '/bbs/Beauty/M.1577801740.A.E41.html'})+1]
    articles_popular_final=articles_popular[articles_popular.index({'date': ' 102', 'title': '[正妹] 泉里香 海賊王娜美真人版廣告 出演', 'href': '/bbs/Beauty/M.1546412589.A.A27.html'}):articles_popular.index({'date': '1228', 'title': '[正妹] 百大美女第一名', 'href': '/bbs/Beauty/M.1577496908.A.F99.html'})+1]
    #寫入檔案
    with open('all_articles.txt', 'w') as filehandle:
        for items in articles_final:
            filehandle.write('%s\n' %','.join([items['date'],items['title'],ppturl+items['href']]))

    with open('all_popular.txt', 'w') as filehandle:
        for items in articles_popular_final:
            filehandle.write('%s\n' %','.join([items['date'],items['title'],ppturl+items['href']]))

In [ ]:
def priority(d):
        num_list=[d[i][1] for i in range(10)]
        if len(num_list) == len(set(num_list)):
            return d
        else:
            name=[]
            index_list=[]
            for elem in set(num_list):
                #對每個有重複值的做排序
                if num_list.count(elem) > 1:
                    for i in range(10):
                        if d[i][1]==elem:
                            name.append(d[i][0])
                            index_list.append(i)
                    name=sorted(name, key=lambda x:x[0])
                    index_list=sorted(index_list)
                    for j in range(len(index_list)):
                        d[index_list[j]]=(name[j],elem)
            return d
                    

In [ ]:
#hw1-2 push
def push(start_date, end_date):
    all_like=0
    all_boo=0
    like={}
    boo={}
    articles=[]
    with open('all_articles.txt', 'r') as f:
        for line in f.readlines():
            if int(line.strip().split(',')[0])>=start_date and int(line.strip().split(',')[0])<=end_date:
                articles.append(line.strip())
    for i in articles:
        article=i.split(',')
        rq=rs.get(article[2])
        objSoup=BeautifulSoup(rq.text, 'lxml')
        if '※ 發信站' not in content:
            continue
        content=objSoup.find('div','bbs-screen bbs-content').find_all('div','push')
        for j in content:
            if j.find_all('span')[0].text.strip()=='推':
                all_like+=1
                x=like.get(j.find_all('span')[1].text,0)
                like[j.find_all('span')[1].text] = x+1
            elif j.find_all('span')[0].text.strip()=='噓':
                all_boo+=1
                x=boo.get(j.find_all('span')[1].text,0)
                boo[j.find_all('span')[1].text] = x+1
    like_10=sorted(like.items(), key=lambda x:x[1],reverse=True)[:10]
    boo_10=sorted(boo.items(), key=lambda x:x[1],reverse=True)[:10]
    like_10=priority(like_10)
    boo_10=priority(boo_10)
    with open('push[%s].txt'%'-'.join([str(start_date),str(end_date)]), 'w') as filehandle:
        filehandle.write('all like: %s\n' %all_like)
        filehandle.write('all boo: %s\n' %all_boo)
        for items in range(10):
            filehandle.write('like #%s: %s %s\n' %(items+1,like_10[items][0],like_10[items][1]))
        for items in range(10):
            filehandle.write('boo #%s: %s %s\n' %(items+1,boo_10[items][0],boo_10[items][1]))

In [ ]:
#hw1-3 popular
def popular(start_date, end_date):
    num_article=0
    pic_url=[]
    articles=[]
    with open('all_popular.txt', 'r') as f:
        for line in f.readlines():
            if int(line.strip().split(',')[0])>=start_date and int(line.strip().split(',')[0])<=end_date:
                articles.append(line.strip())
    for i in articles:
        article=i.split(',')
        rq=rs.get(article[2])
        objSoup=BeautifulSoup(rq.text, 'lxml')
        content=objSoup.find('div','bbs-screen bbs-content').getText()
        if '※ 發信站' not in content:
            continue
        content=objSoup.find('div','bbs-screen bbs-content').find_all('a')
        for j in content:
            if '.jpg' in j.text:
                pic_url.append(j.text)
                num_article+=1
            elif '.jpeg' in j.text:
                pic_url.append(j.text)
                num_article+=1
            elif '.png' in j.text:
                pic_url.append(j.text)
                num_article+=1
            elif '.gif' in j.text:
                pic_url.append(j.text)
                num_article+=1
    with open('popular[%s].txt'%'-'.join([str(start_date),str(end_date)]), 'w') as filehandle:
        filehandle.write('number of popular articles: %s\n' %num_article)
        for items in range(len(pic_url)):
            filehandle.write('%s\n' %pic_url[items])

In [ ]:
#hw1-4 keyword
def keyword(keyword,start_date, end_date):
    num_article=0
    pic_url=[]
    articles=[]
    with open('all_articles.txt', 'r') as f:
        for line in f.readlines():
            if int(line.strip().split(',')[0])>=start_date and int(line.strip().split(',')[0])<=end_date:
                articles.append(line.strip())
    for i in articles:
        article=i.split(',')
        rq=rs.get(article[2])
        objSoup=BeautifulSoup(rq.text, 'lxml')
        content=objSoup.find('div','bbs-screen bbs-content').getText()
        if '※ 發信站' in content:
            content=content.split('※ 發信站')[0]
        if keyword in content:
            content=objSoup.find('div','bbs-screen bbs-content').find_all('a')
            for j in content:
                if '.jpg' in j.text:
                    pic_url.append(j.text)
                elif '.jpeg' in j.text:
                    pic_url.append(j.text)
                elif '.png' in j.text:
                    pic_url.append(j.text)
                elif '.gif' in j.text:
                    pic_url.append(j.text)
    with open('keyword(%s)[%s].txt'%(keyword,'-'.join([str(start_date),str(end_date)])), 'w') as filehandle:
        for items in range(len(pic_url)):
            filehandle.write('%s\n' %pic_url[items])

In [ ]:
#train model!!!!
#data
#把題目資訊擷取下來
def catch_image(content):
    image_list=[]
    image_popular_list=[]
    objSoup=BeautifulSoup(content, 'lxml')
    pttdivs=objSoup.find_all('div','r-ent')
    a=0
    for p in pttdivs:
        #print(p.find('div','nrec').text)
        if p.find('a') and '公告' not in p.find('a').text:
            if p.find('div','nrec').text==''or p.find('div','nrec').text=='X1':
                a=1
                #print(a,index_page)
            #print(a)
            if a==0 and (p.find('div','nrec').text=='爆'or int(p.find('div','nrec').text)>=35):
                #print('popular',p.find('div','nrec').text)
                title=p.find('a').text
                href=p.find('a')['href']
                rq=rs.get(ppturl+href)
                objSoup_content=BeautifulSoup(rq.text, 'lxml')
                content=objSoup_content.find('div','bbs-screen bbs-content').getText()
                if '※ 發信站' not in content:
                    continue
                content=objSoup_content.find('div','bbs-screen bbs-content').find_all('a')
                pic_url=[]
                for j in content:
                    if '.jpg' in j.text:
                        pic_url.append(j.text)
                image_popular_list.append({'title':title,'href':href,'pic':pic_url})
                a=0
            else:
                
                title=p.find('a').text
                href=p.find('a')['href']
                rq=rs.get(ppturl+href)
                objSoup_content=BeautifulSoup(rq.text, 'lxml')
                content=objSoup_content.find('div','bbs-screen bbs-content').getText()
                if '※ 發信站' not in content:
                    continue
                content=objSoup_content.find('div','bbs-screen bbs-content').find_all('a')
                pic_url=[]
                for j in content:
                    if '.jpg' in j.text:
                        pic_url.append(j.text)
                image_list.append({'title':title,'href':href,'pic':pic_url})
                a=0
            
    return image_list,image_popular_list,objSoup

def crawl_image():
    b=1
    articles=[]
    articles_popular=[]
    while b==1:
        global index_page
        r=rs.get(index_page)
        articles_list,articles_popular_list,objSoup=catch_image(r.content)
        articles.extend(articles_list) 
        articles_popular.extend(articles_popular_list)
        div_page=objSoup.find('div',"btn-group btn-group-paging")
        next_page=div_page.find_all('a')[2]['href']
        index_page=ppturl+next_page
        if len(articles)>1500:
            b=0
    with open('image.txt', 'w') as filehandle:
        for items in articles:
            for pic in items['pic']:
                filehandle.write('%s\n' %','.join([items['title'],ppturl+items['href'],pic]))

    with open('popular_image.txt', 'w') as filehandle:
        for items in articles_popular:
            for pic in items['pic']:
                filehandle.write('%s\n' %','.join([items['title'],ppturl+items['href'],pic]))
crawl_image()
    


In [ ]:
with open('image_all.txt', 'r') as f:
    num=0
    for line in f.readlines():
        href=line.split('\t')[0]
        
        if href.split('.')[-1]!='jpg':
            continue
        target=line.split('\t')[1].strip()
        if target=='0':
            try:
                img = requests.get(href)
            except requests.exceptions.ConnectionError:
                r.status_code = "Connection refused"
            with open('./images/0/'+str(num)+'.jpg','wb') as f:
                f.write(img.content)
            num+=1
        else:
            try:
                img = requests.get(href)
            except requests.exceptions.ConnectionError:
                r.status_code = "Connection refused"
            with open('./images/1/'+str(num)+'.jpg','wb') as f:
                f.write(img.content)
            num+=1




    


In [22]:
file_list=os.walk(path)
print(file_list)

<generator object walk at 0x7fcf82ba9550>


In [26]:
#刪除存錯副檔名的資料
path='./images/0/'
for pth, dirs, files in os.walk(path):

    for file in files:

        try :
            Image.open(path+file)
            print('success open')
        except OSError:
            os.remove(path+file)
            print('remove')

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
remove
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
succe

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
remove
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
succe

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open


In [13]:
#刪除存錯副檔名的資料
path='./images/1/'
file_list=os.listdir(path)
for file in file_list:
    size=os.path.getsize(path+file)
    try :
        Image.open(path+file).convert('RGB')
        print('success open')
    except OSError:
        os.remove(path+file)
        print('remove')

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open
success open

In [ ]:
#刪除空資料
path='./images/0/'
file_list=os.listdir(path)
for file in file_list:
    size=os.path.getsize(path+file)
    if size == 0:
        os.remove(path+file)
        print('delete')
    else:
        print('not empty')

In [2]:
img_data = torchvision.datasets.ImageFolder('./images',
                                            transform=transforms.Compose([
                                                transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
                                            )
batch_size = 50
train_size = int(0.8 * len(img_data))
val_size = len(img_data) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(img_data, [train_size, val_size])
dataloaders = {}
dataloaders['train_data'] = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           num_workers=4)
dataloaders['valid_data'] = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                                num_workers=4)




In [3]:

import torch
import torch.nn as nn


class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        x38 = self.fc8(x37)
        return x38

def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [4]:
model=vgg_face_dag('vgg_face_dag.pth')

In [5]:

# Freeze parameters so we don't backprop through them
def freeze_parameters(root, freeze=True):
    [param.requires_grad_(not freeze) for param in root.parameters()]
    
# Create a new classifier
def create_classifier(input_size, output_size, hidden_layers=[], dropout=0.5,
                      activation=nn.RReLU(), output_function=nn.LogSoftmax(dim=1)):
    dict = OrderedDict()
    
    if len(hidden_layers) == 0:
        dict['layer0'] = nn.Linear(input_size, output_size)

    else:
        
        dict['layer0'] = nn.Linear(input_size, hidden_layers[0])
        if activation:
            dict['activ0'] = activation
        if dropout:
            dict['drop_0'] = nn.Dropout(dropout)
        
        #for layer_in, layer_out in range(len(hidden_layers)):
        for layer, layer_in in enumerate(zip(hidden_layers[:-1],hidden_layers[1:])):
            dict['layer'+str(layer+1)] = nn.Linear(layer_in[0],layer_in[1])
            if activation:
                dict['activ'+str(layer+1)] = activation
            if dropout:
                dict['drop_'+str(layer+1)] = nn.Dropout(dropout)
            
        dict['output'] = nn.Linear(hidden_layers[-1], output_size)

    if output_function:
        dict['output_function'] = output_function
    
    return nn.Sequential(dict)

# Build and train your network
def create_network(model_name='resnet18', output_size=2, hidden_layers=[1000]):
    if model_name == 'resnet18':
        # Download the model
        model = models.resnet18(pretrained=True)
        # Replace the model classifier
        model.fc = create_classifier(512, output_size, hidden_layers)
        
        return model
        
    if model_name == 'resnet152':
        # Download the model
        model = models.resnet152(pretrained=True)
        # Replace the model classifier
        model.fc = create_classifier(512, output_size, hidden_layers)
        
        return model
        
    return None

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'Running on: {str(device).upper()}')

Running on: CPU


In [7]:
gpustat

NameError: name 'gpustat' is not defined

In [68]:
# Define hyperparameters
model_name = 'resnet18'
output_size = 2
hidden_layers = [10]

model = create_network(model_name, output_size, hidden_layers)

# Move model to device
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [65]:
def train(epochs, model, optimizers, lr_scheduler=None,
          dataloaders=dataloaders, state_dict=None,
          checkpoint_path="checkpoint.pt", accuracy_target=None,
          show_graphs=True):

    if state_dict == None:
        state_dict = {
            'elapsed_time': 0,
            'trace_log': [],
            'trace_train_loss': [],
            'trace_train_lr': [],
            'valid_loss_min': np.Inf,
            'trace_valid_loss': [],
            'trace_accuracy': [],
            'epochs_trained': 0}
        state_dict['trace_log'].append('PHASE ONE')


    for epoch in range(1,epochs+1):
        
        try:
            lr_scheduler.step() # if instance of _LRScheduler
        except TypeError:
            try:
                if lr_scheduler.min_lrs[0] == lr_scheduler.optimizer.param_groups[0]['lr']:
                    break
                lr_scheduler.step(valid_loss) # if instance of ReduceLROnPlateau
            except NameError: # valid_loss is not defined yet
                lr_scheduler.step(np.Inf)
        except:
            pass # do nothing

        epoch_start_time = time.time()
        #####################
        #       TRAIN       #
        #####################
        train_loss = 0
        model.train()
        for images, labels in dataloaders['train_data']:
            # Move tensors to device
            images, labels = images.to(device), labels.to(device)
            
            # Clear optimizers
            [opt.zero_grad() for opt in optimizers]

            # Pass train batch through model feed-forward
            output = model(images)
            
            # Calculate loss for this train batch
            batch_loss = criterion(output, labels-1)
            # Do the backpropagation
            batch_loss.backward()

            # Optimize parameters
            [opt.step() for opt in optimizers]

            # Track train loss
            train_loss += batch_loss.item()*len(images)

        # Track how many epochs has already run
        state_dict['elapsed_time'] += time.time()-epoch_start_time
        state_dict['epochs_trained'] += 1

        #####################
        #      VALIDATE     #
        #####################
        valid_loss = 0
        accuracy = 0
        top_class_graph = []
        labels_graph = []
        # Set model to evaluation mode
        model.eval()
        with torch.no_grad():
            for images, labels in dataloaders['valid_data']:
                labels_graph.extend( labels )

                # Move tensors to device
                images, labels = images.to(device), labels.to(device)

                # Get predictions for this validation batch
                output = model(images)
                
                # Calculate loss for this validation batch
                batch_loss = criterion(output, labels-1)
                # Track validation loss
                valid_loss += batch_loss.item()*len(images)

                # Calculate accuracy
                output = torch.exp(output)
                top_ps, top_class = output.topk(1, dim=1)
                top_class_graph.extend( top_class.view(-1).to('cpu').numpy() )
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*len(images)

        #####################
        #     PRINT LOG     #
        #####################
        
        # calculate average losses
        train_loss = train_loss/len(dataloaders['train_data'].dataset)
        valid_loss = valid_loss/len(dataloaders['valid_data'].dataset)
        accuracy = accuracy/len(dataloaders['valid_data'].dataset)

        state_dict['trace_train_loss'].append(train_loss)
        try:
            state_dict['trace_train_lr'].append(lr_scheduler.get_lr()[0])
        except:
            state_dict['trace_train_lr'].append(
                optimizers[0].state_dict()['param_groups'][0]['lr'])
        state_dict['trace_valid_loss'].append(valid_loss)
        state_dict['trace_accuracy'].append(accuracy)

        # print training/validation statistics 
        log = 'Epoch: {}: \
               lr: {:.8f}\t\
               Training Loss: {:.6f}\t\
               Validation Loss: {:.6f}\t\
               Validation accuracy: {:.2f}%\t\
               Elapsed time: {:.2f}'.format(
                    state_dict['epochs_trained'],
                    state_dict['trace_train_lr'][-1],
                    train_loss,
                    valid_loss,
                    accuracy*100,
                    state_dict['elapsed_time']
                    )
        state_dict['trace_log'].append(log)
        print(log)

        # save model if validation loss has decreased
        if valid_loss <= state_dict['valid_loss_min']:
            print('Validation loss decreased: \
                  ({:.6f} --> {:.6f}).   Saving model ...'
                  .format(state_dict['valid_loss_min'],valid_loss))

            checkpoint = {'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizers[0].state_dict(),
                          'training_state_dict': state_dict}
            if lr_scheduler:
                checkpoint['lr_scheduler_state_dict'] = lr_scheduler.state_dict()

            torch.save(checkpoint, checkpoint_path)
            state_dict['valid_loss_min'] = valid_loss
        '''
        if show_graphs:
            plt.figure(figsize=(25,8))
            plt.plot(np.array(labels_graph), 'k.')
            plt.plot(np.array(top_class_graph), 'r.')
            plt.show()

            plt.figure(figsize=(25,5))
            plt.subplot(1,2,1)
            plt.plot(np.array(state_dict['trace_train_loss']), 'b', label='train loss')
            plt.plot(np.array(state_dict['trace_valid_loss']), 'r', label='validation loss')
            plt.plot(np.array(state_dict['trace_accuracy']), 'g', label='accuracy')

            plt.subplot(1,2,2)
            plt.plot(np.array(state_dict['trace_train_lr']), 'b', label='train loss')

            plt.show()

    '''
        # stop training loop if accuracy_target has been reached
        if accuracy_target and state_dict['trace_accuracy'][-1] >= accuracy_target:
            break

    return state_dict


In [31]:
def load_model(checkpoint_path, state_dict):
    try:
        checkpoint = torch.load(checkpoint_path)
        state_dict = checkpoint['training_state_dict']
        model.load_state_dict(checkpoint['model_state_dict'])
        fc_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    except:
        pass

    return state_dict

In [ ]:
# Training strategy

# Define a loss function
criterion = nn.NLLLoss() # Cross-Entropy (LogSoftmax and NLLLoss)

# Define how many times each phase will be running
PHASE_ONE = 50
PHASE_TWO = 20
PHASE_THREE = 10

TEST = False

# Define the phases
if PHASE_ONE > 0:
    freeze_parameters(model)
    freeze_parameters(model.fc8, False)

    fc_optimizer = optim.Adagrad(model.fc8.parameters(), lr=0.01, weight_decay=0.001)
    optimizers = [fc_optimizer]

    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(fc_optimizer, mode='min',
                                                   factor=0.1, patience=5,
                                                   threshold=0.01, min_lr=0.00001)
                                                           
    checkpoint_path = "checkpoint_phase_one.pt"
    
    state_dict = train(PHASE_ONE, model, optimizers, lr_scheduler=lr_scheduler,
                       state_dict=None, accuracy_target=None,
                       checkpoint_path=checkpoint_path)

    print(*state_dict['trace_log'], sep="\n")
    
    state_dict = load_model(checkpoint_path, state_dict)

if PHASE_TWO > 0:
    state_dict['trace_log'].append('PHASE TWO')

    freeze_parameters(model, False)

    conv_optimizer = optim.Adagrad(model.parameters(), lr=0.0001, weight_decay=0.001)
    optimizers = [fc_optimizer, conv_optimizer]

    checkpoint_path = "checkpoint_phase_two.pt"
    
    state_dict = train(PHASE_TWO, model, optimizers, lr_scheduler=None,
                       state_dict=state_dict, accuracy_target=None,
                       checkpoint_path=checkpoint_path)

    print(*state_dict['trace_log'], sep="\n")

    state_dict = load_model(checkpoint_path, state_dict)

if PHASE_THREE > 0:
    state_dict['trace_log'].append('PHASE THREE')

    freeze_parameters(model)
    freeze_parameters(model.fc8, False)

    optimizers = [fc_optimizer]
    
    lr_scheduler = optim.lr_scheduler.MultiStepLR(fc_optimizer, milestones=[0], gamma=0.01)
        
    checkpoint_path = "checkpoint_phase_three.pt"
    
    state_dict = train(PHASE_THREE, model, optimizers, lr_scheduler=lr_scheduler,
                       state_dict=state_dict, accuracy_target=None,
                       checkpoint_path=checkpoint_path)

    print(*state_dict['trace_log'], sep="\n")

    state_dict = load_model(checkpoint_path, state_dict)
if TEST:
    test_model()

In [ ]:
# Save the checkpoint 
def save_checkpoint(checkpoint_path='checkpoint.pt'):
    model.to('cpu')
    checkpoint = {'model_name': model_name,
                  'output_size': output_size,
                  'hidden_layers': hidden_layers,
                  'model_state_dict': model.state_dict(),
                  'class_to_idx': class_to_idx,
                  'cat_label_to_name': cat_label_to_name}

    torch.save(checkpoint, checkpoint_path)

In [ ]:
save_checkpoint('checkpoint.pt')

In [ ]:
# Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(checkpoint_path='checkpoint.pt'):
    checkpoint = torch.load(checkpoint_path)

    model_name = checkpoint['model_name']
    output_size = checkpoint['output_size']
    hidden_layers = checkpoint['hidden_layers']
    
    model = create_network(model_name=model_name,
                         output_size=output_size, hidden_layers=hidden_layers)
   
    model.load_state_dict(checkpoint['model_state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    model.cat_label_to_name = checkpoint['cat_label_to_name']
    
    return model

In [ ]:
model = load_checkpoint('checkpoint.pt')

model.to(device)

In [ ]:
def predict(image_path, model, topk=1):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # Implement the code to predict the class from an image file
    image = Image.open(image_path)
    image = process_image(image)
    
    with torch.no_grad():
        model.eval()
        
        image = image.view(1,3,224,224)
        image = image.to(device)
        
        predictions = model.forward(image)
        
        predictions = torch.exp(predictions)
        top_ps, top_class = predictions.topk(topk, dim=1)
    
    return top_ps, top_class

In [ ]:
probs, classes = predict(image_path, model)
print(probs)
print(classes)

In [ ]:
#HW 1-2
def imgfilelistname(file):
    img=[]
    with open(file, 'r') as f:
        for line in f.readlines():
            img.append(line.strip())
    # load all images in a directory
    target = []
    for filename in img:
        # load image
        img_data = Image.open(filename).convert('RGB')
        # store loaded image
        transform = trandforms.Compose([trandforms.Resize((224, 224)), trandforms.ToTensor()])
        image_tensor = transform(img_data)
        image_tensor = image_tensor.unsqueeze(0)
        model=torchvision.models.resnet18(pretrained=True, progress=True, num_classes=2)
        model.eval()
        output=model(image_tensor)
        target.append(output.max().index) 
    

In [ ]:
a=1
b=3
c=3
if a==2 and (b==3 or c==3):
    print('success')
else: print('fail')

In [ ]:
{'l':[1,2,3]}

In [ ]:
if __name__ == '__main__':
    if sys.argv[1]=='crawl':
        crawl()
    elif sys.argv[1]=='push':
        push(sys.argv[2],sys.argv[3])
    elif sys.argv[1]=='popular':
        popular(sys.argv[2],sys.argv[3])
    elif sys.argv[1]=='keyword':
        keyword(sys.argv[2],sys.argv[3],sys.argv[4])

In [ ]:
python Homework1.py crawl